<a href="https://colab.research.google.com/github/dTenebrae/nlp/blob/main/lesson8/hw8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Задание

На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для задачи сантимент анализа на данных с вебинара
1. построить свёрточные архитектуры
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN и (RNN -> CNN)
4. сделать выводы что получилось лучше

In [1]:
from IPython.display import clear_output

In [2]:
!pip install stop_words
!pip install pymorphy2
clear_output()

In [3]:
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, MaxPooling1D, Flatten
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.metrics import categorical_crossentropy
from keras.callbacks import EarlyStopping

import warnings
warnings.simplefilter("ignore")

In [4]:
df_train = pd.read_csv("/content/drive/MyDrive/csv/nlp_hw7_data/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/csv/nlp_hw7_data/test.csv")
df_val = pd.read_csv("/content/drive/MyDrive/csv/nlp_hw7_data/val.csv")

In [5]:
result = []

In [6]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [7]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [8]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [9]:
y_train = df_train['class'].values
y_val = df_val['class'].values

### Варианты рекуррентных сетей

**Простая**

In [10]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
early_stopping=EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 15s 40ms/step - loss: 0.5695 - accuracy: 0.6884 - val_loss: 0.4942 - val_accuracy: 0.7529
Epoch 2/10
319/319 [==============================] - 12s 39ms/step - loss: 0.2864 - accuracy: 0.8839 - val_loss: 0.5541 - val_accuracy: 0.7422
Epoch 3/10
319/319 [==============================] - 12s 39ms/step - loss: 0.0784 - accuracy: 0.9729 - val_loss: 0.7871 - val_accuracy: 0.7249
Epoch 4/10
319/319 [==============================] - 13s 40ms/step - loss: 0.0315 - accuracy: 0.9897 - val_loss: 0.9871 - val_accuracy: 0.7190


In [12]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=0)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])
result.append(score)



Test loss: 0.5158835649490356
Test accuracy: 0.7372922301292419


**LSTM**

In [13]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
early_stopping=EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)  

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 37s 108ms/step - loss: 0.5556 - accuracy: 0.7051 - val_loss: 0.4936 - val_accuracy: 0.7528
Epoch 2/10
319/319 [==============================] - 34s 107ms/step - loss: 0.3288 - accuracy: 0.8638 - val_loss: 0.5429 - val_accuracy: 0.7452
Epoch 3/10
319/319 [==============================] - 34s 107ms/step - loss: 0.1629 - accuracy: 0.9391 - val_loss: 0.6759 - val_accuracy: 0.7334
Epoch 4/10
319/319 [==============================] - 35s 108ms/step - loss: 0.1017 - accuracy: 0.9630 - val_loss: 0.7813 - val_accuracy: 0.7288


In [15]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=0)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])
result.append(score)



Test loss: 0.5072261691093445
Test accuracy: 0.7461975812911987


**GRU**

In [16]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
early_stopping=EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)  

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 33s 97ms/step - loss: 0.5528 - accuracy: 0.7089 - val_loss: 0.4887 - val_accuracy: 0.7559
Epoch 2/10
319/319 [==============================] - 31s 96ms/step - loss: 0.3131 - accuracy: 0.8709 - val_loss: 0.5471 - val_accuracy: 0.7494
Epoch 3/10
319/319 [==============================] - 31s 96ms/step - loss: 0.1383 - accuracy: 0.9493 - val_loss: 0.6776 - val_accuracy: 0.7392
Epoch 4/10
319/319 [==============================] - 30s 95ms/step - loss: 0.0802 - accuracy: 0.9718 - val_loss: 0.8557 - val_accuracy: 0.7330


In [18]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=0)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])
result.append(score)



Test loss: 0.516929030418396
Test accuracy: 0.7403782606124878


### Сверточная сеть

In [19]:
embedding_layer = Embedding(input_dim=word_count, output_dim=30, input_length=training_length)

In [20]:
sequence_input = Input(shape=(training_length,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu', data_format='channels_first')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu', data_format='channels_first')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu', data_format='channels_first')(x)
x = GlobalMaxPool1D()(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(1, activation='sigmoid')(x)

model = Model(sequence_input, preds)
model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
early_stopping=EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)  

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 18s 9ms/step - loss: 0.5556 - accuracy: 0.6955 - val_loss: 0.4941 - val_accuracy: 0.7538
Epoch 2/10
319/319 [==============================] - 3s 8ms/step - loss: 0.2990 - accuracy: 0.8745 - val_loss: 0.5401 - val_accuracy: 0.7461
Epoch 3/10
319/319 [==============================] - 3s 8ms/step - loss: 0.0908 - accuracy: 0.9665 - val_loss: 0.7248 - val_accuracy: 0.7316
Epoch 4/10
319/319 [==============================] - 3s 8ms/step - loss: 0.0311 - accuracy: 0.9897 - val_loss: 0.9983 - val_accuracy: 0.7310


In [22]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=0)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])
result.append(score)



Test loss: 0.5073196291923523
Test accuracy: 0.7455363273620605


### Гибридная сеть

In [23]:
model = Sequential()
model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
early_stopping=EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)  

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 5s 9ms/step - loss: 0.5564 - accuracy: 0.6991 - val_loss: 0.4895 - val_accuracy: 0.7568
Epoch 2/10
319/319 [==============================] - 2s 8ms/step - loss: 0.3000 - accuracy: 0.8742 - val_loss: 0.5351 - val_accuracy: 0.7468
Epoch 3/10
319/319 [==============================] - 2s 8ms/step - loss: 0.1182 - accuracy: 0.9559 - val_loss: 0.6960 - val_accuracy: 0.7360
Epoch 4/10
319/319 [==============================] - 2s 7ms/step - loss: 0.0573 - accuracy: 0.9801 - val_loss: 0.8511 - val_accuracy: 0.7336


In [25]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=0)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])
result.append(score)



Test loss: 0.5318854451179504
Test accuracy: 0.7314288020133972


Значимого прироста в качестве гибридная сеть не дала

**Посмотрим на результаты работы сетей**

In [30]:
networks = ['simpleRNN', 'LSTM', 'GRU', 'CNN', 'Hybrid']
pd.DataFrame(result, index=networks, columns=['loss', 'accuracy'])

,loss,accuracy
simpleRNN,0.515884,0.737292
LSTM,0.507226,0.746198
GRU,0.516929,0.740378
CNN,0.507320,0.745536
Hybrid,0.531885,0.731429


Как можно увидеть, значимых отличий не наблюдается. Но итоговый лучший результат у LSTM и CNN